# Humpback Whale Identification - CNN with Keras

In [1]:
import gc
import os

import numpy as np
import pandas as pd
import progressbar

import matplotlib.pyplot as plt
import matplotlib.image as mplimg
from matplotlib.pyplot import imshow

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

from keras import layers
from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout
from keras.models import Model

import keras.backend as K
from keras.models import Sequential

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

Using TensorFlow backend.


In [2]:
train_df = pd.read_csv("../input/train.csv")
train_df.head()

,Image,Id
0,0000e88ab.jpg,w_f48451c
1,0001f9222.jpg,w_c3d896a
2,00029d126.jpg,w_20df2c5
3,00050a15a.jpg,new_whale
4,0005c1ef8.jpg,new_whale


In [3]:
import cv2

# From https://stackoverflow.com/questions/39308030/how-do-i-increase-the-contrast-of-an-image-in-python-opencv#41075028
def increaseContrast(path):
    #-----Reading the image-----------------------------------------------------
    img = cv2.imread(path, 1)
    #-----Converting image to LAB Color model-----------------------------------
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    #-----Splitting the LAB image to different channels-------------------------
    l, a, b = cv2.split(lab)
    #-----Applying CLAHE to L-channel-------------------------------------------
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    #-----Merge the CLAHE enhanced L-channel with the a and b channel-----------
    limg = cv2.merge((cl, cl, cl)) # (cl, a, b)
    #-----Converting image from LAB Color model to RGB model--------------------
    final = limg #final = cv2.cvtColor(limg, cv2.COLOR_LAB2BGR)
    #-----Resize----------------------------------------------------------------
    resized = cv2.resize(final, dsize=(128, 128), interpolation=cv2.INTER_CUBIC)
    return resized

In [ ]:
def prepareImages(data, m, dataset):
    #print("Preparing images")
    #print(m)
    X_train = np.zeros((m, 128, 128, 3))
    count = 0
    
    for fig in progressbar.progressbar(data['Image']):
        # load images into images of size 128x128x3
        x = increaseContrast("../input/"+dataset+"/"+fig) # added by Yvan
        x = preprocess_input(x)
        X_train[count] = x
        count += 1
    
    return X_train

In [ ]:
def prepare_labels(y):
    values = np.array(y)
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(values)
    #print(integer_encoded)

    onehot_encoder = OneHotEncoder(sparse=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    onehot_encoded = onehot_encoder.fit_transform(integer_encoded)
    #print(onehot_encoded)

    y = onehot_encoded
    #print(y.shape)
    return y, label_encoder

In [ ]:
X = prepareImages(train_df, train_df.shape[0], "train")
X /= 255

100% (25361 of 25361) |##################| Elapsed Time: 0:10:50 Time:  0:10:50


In [ ]:
y, label_encoder = prepare_labels(train_df['Id'])

/opt/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [ ]:
y.shape

(25361, 5005)

# Change model starting from here

## Train

In [ ]:
model = Sequential()

model.add(Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0', input_shape = (128, 128, 3)))

model.add(BatchNormalization(axis = 3, name = 'bn0'))
model.add(Activation('relu'))

model.add(MaxPooling2D((2, 2), name='max_pool'))
model.add(Conv2D(64, (3, 3), strides = (1,1), name="conv1"))
model.add(Activation('relu'))
model.add(AveragePooling2D((3, 3), name='avg_pool'))

model.add(Flatten())
model.add(Dense(500, activation="relu", name='rl'))
model.add(Dropout(0.8))
model.add(Dense(y.shape[1], activation='softmax', name='sm'))

model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv0 (Conv2D)               (None, 122, 122, 32)      4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 122, 122, 32)      128       
_________________________________________________________________
activation_1 (Activation)    (None, 122, 122, 32)      0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 61, 61, 32)        0         
_________________________________________________________________
conv1 (Conv2D)               (None, 59, 59, 64)        18496     
_________________________________________________________________
activation_2 (Activation)    (None, 59, 59, 64)        0         
_________________________________________________________________
avg_pool (AveragePooling2D)  (None, 19, 19, 64)        0         
__________

In [ ]:
history = model.fit(X, y, epochs=100, batch_size=100, verbose=1)
gc.collect()

Epoch 1/100
25361/25361 [==============================] - 47s 2ms/step - loss: 6.2264 - acc: 0.3780
Epoch 2/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.9523 - acc: 0.3811
Epoch 3/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.8696 - acc: 0.3811
Epoch 4/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.7849 - acc: 0.3811
Epoch 5/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.7387 - acc: 0.3811
Epoch 6/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.7039 - acc: 0.3811
Epoch 7/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.6558 - acc: 0.3811
Epoch 8/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.5904 - acc: 0.3811
Epoch 9/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.5278 - acc: 0.3811
Epoch 10/100
25361/25361 [==============================] - 45s 2ms/step - loss: 5.4629 - a

In [ ]:
plt.plot(history.history['acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.show()

## Test

In [ ]:
test = os.listdir("../input/test/")
print(len(test))

In [ ]:
col = ['Image']
test_df = pd.DataFrame(test, columns=col)
test_df['Id'] = ''

In [ ]:
X = prepareImages(test_df, test_df.shape[0], "test")
X /= 255

In [ ]:
predictions = model.predict(np.array(X), verbose=1)

In [ ]:
for i, pred in enumerate(predictions):
    test_df.loc[i, 'Id'] = ' '.join(label_encoder.inverse_transform(pred.argsort()[-5:][::-1]))

In [ ]:
test_df.head(10)
test_df.to_csv('keras-cnn-starter-high-contrast.csv', index=False) #> Score = 289

In [ ]:
!kaggle competitions submit -c humpback-whale-identification -f "keras-cnn-starter-high-contrast.csv" -m "Keras CNN starter + High Contrast"